In [1]:
import numpy as np
from BPmodule.BPGen import PROBLEM2, PROBLEM3, PROBLEM4, PROBLEM5
from BPnumba.GeneticOperators import CreateHeuristicPob,Ind
from BPnumba.BPPdat import NumDBLF,create_Bin,Bin,Placement,Overlap,IterateDBLF
from BPnumba.PqueueNumba import CreatePriorityQueue
from numba import njit, objmode, prange
import time
from numba.typed import List as NumbaList
from typing import List

In [2]:
P2A1 = PROBLEM2[0]
P2A2 = PROBLEM2[1]
P3A1 = PROBLEM3[0]
P3A2 = PROBLEM3[1]
P4A1 = PROBLEM4[0]
P4A2 = PROBLEM4[1]
P5A1 = PROBLEM5[0]
P5A2 = PROBLEM5[1]

In [3]:
contenedor = P2A1[0][0]
datos = P2A1[1]
n = len(datos)
sameInstance = CreateHeuristicPob(20,datos,True)
n

29

In [4]:
ind = Ind(NumbaList(sameInstance[1]))
ind.genome

ListType[int64]([3, 2, 5, 4, 1, 26, 28, 27, 29, 19, 21, 18, 20, 6, 8, 7, 9, 10, 12, 11, 13, 22, 24, 23, 25, 15, 17, 14, 16, ...])

In [5]:
@njit
def DBLF(bin:Bin, itemsToPack:List[int], BoxesData:List[List[int]]):
    lstP = CreatePriorityQueue(NumbaList([0,2,1]))
    lstP.push(NumbaList([0,0,0]))
    for i in np.arange(len(itemsToPack)):
        for j in np.arange(lstP.size()):
            pt = lstP.getPt(j)
            boxID = itemsToPack[i]
            itemV = BoxesData[boxID-1]
            if  Placement(bin.dimensions,NumbaList([pt[0]+itemV[0],pt[1]+itemV[1],pt[2]+itemV[2]])):
                intersect = Overlap(pt,boxID,BoxesData, bin.getBoxes(),bin.getPositions())
                if intersect:
                    continue
                else:
                    lstP.delPt(j)
                    IterateDBLF(pt,boxID,BoxesData,bin.getBoxes(),bin.getPositions())
                    bin.addBox(boxID,pt,itemV)
                    for k in np.arange(3):
                        if pt[k] + itemV[k] < bin.dimensions[k]:
                            pt[k] += itemV[k]
                            lstP.push(NumbaList([pt[0],pt[1],pt[2]]))
                            pt[k] -= itemV[k]
                    lstP.eliminateDuplicates()                   
                    break
    lstP.printQ()

In [6]:
bin = create_Bin(NumbaList(contenedor))
DBLF(bin,ind.genome,datos)
print(bin.getLoadVol()/np.prod(contenedor))
print(bin.getN(),bin.getBoxes())

[39, 49, 48, ...]
[39, 49, 49, ...]
[45, 49, 48, ...]
[45, 49, 49, ...]
[49, 42, 48, ...]
[49, 42, 49, ...]
0.999808
27 [3, 2, 5, 4, 1, 26, 28, 27, 29, 19, 21, 18, 20, 6, 8, 7, 10, 12, 11, 22, 24, 23, 25, 15, 17, 14, 16, ...]
